<a href="https://colab.research.google.com/github/agstnhrvy/UPLB-DCE-Thesis/blob/main/Thesis_Code_%5BOrganized%5D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [23]:
# Thesis Project
# To properly install pyswmm and swmm-api to the Python file, select the python interpreter from the venv\Scripts\python.exe then install both libraries using "pip install ..."

!pip install pyswmm
!pip install swmm_api
!pip install swmm5
!pip install openpyxl
import math
import itertools
import os, sys
import swmm5
from pyswmm import Simulation, Links
from pyswmm.swmm5 import SWMMException
from pyswmm.nodes import Nodes
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.section_labels import JUNCTIONS, CONDUITS
from swmm_api.input_file.sections import Conduit
from swmm.toolkit.shared_enum import LinkResult
from google.colab import files
import pandas as pd
import re
import copy

# File Upload

In [24]:
uploaded = files.upload()

inp = SwmmInput("Inlet_Drains_Model1.inp")
out = SwmmOutput("Inlet_Drains_Model1.out")
rpt = SwmmReport("Inlet_Drains_Model1.rpt")
available_sizes = uploaded["Available Sizes.txt"]
print("Uploaded files:", os.listdir())
conduits_df = inp["CONDUITS"]
print(conduits_df)

Saving Available Sizes.txt to Available Sizes.txt
Saving Inlet_Drains_Model1.inp to Inlet_Drains_Model1.inp
Saving Inlet_Drains_Model1.out to Inlet_Drains_Model1.out
Saving Inlet_Drains_Model1.rpt to Inlet_Drains_Model1.rpt
Uploaded files: ['.config', 'Inlet_Drains_Model1.inp', 'Inlet_Drains_Model1.out', '.ipynb_checkpoints', 'Inlet_Drains_Model1.rpt', 'Available Sizes.txt', 'sample_data']
{'Street1': Conduit(name='Street1', from_node='Aux1', to_node='Aux2', length=377.31, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street2': Conduit(name='Street2', from_node='Aux2', to_node='Aux4', length=286.06, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street3': Conduit(name='Street3', from_node='Aux4', to_node='Aux5', length=239.41, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street4': Conduit(name='Street4', from_node='Aux5', to_node='Aux6', len

# Boundary Parameters

In [25]:
peak_flow_dict = {}
peak_flow_results =  {}
peak_vel = {}
adjust_pipe_size = {}
pipe_geometry = {}
conduit_ids = []
link_length = {}
in_offset = {}
out_offset = {}
vel_min = 0.8
vel_max = 5
geom1 = {}
geom2 = {}
geom3 = {}
geom4 = {}
trials = {}
velocity = {}

# Data Frame

In [26]:
conduits = inp['CONDUITS']
xsections = inp['XSECTIONS']
junctions_data = inp['JUNCTIONS']
outfalls_data = inp['OUTFALLS']

# Store results
conduit_info = []

# Accessing Data Dictionary

In [ ]:
for conduit_name, conduit_data in inp["CONDUITS"].items():
  conduit_dict = vars(conduit_data) # Access all data in Conduit
  print (conduit_dict)

print(xsections)  # Print the content of the xsections InpSection object
for xsection_name, xsection_data in xsections.items():
  print(xsection_data)  # Print the name of each parameter in the xsections section

Test 1

In [ ]:
with Simulation("Inlet_Drains_Model1.inp") as sim:
    links = Links(sim)  # Create a Links object
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid  # Get conduit name (link ID)
      conduit_data = link.conduit_statistics  # The 'link' object itself holds the conduit data

      peak_flow = conduit_data["peak_flow"]

      # # If you want to store it in a dictionary:
      # peak_flow_dict = {}  # Initialize an empty dictionary if needed
      # peak_flow_dict[conduit_name] = peak_flow
      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {conduit_data}")
      print(f"Peak Flow: {peak_flow}")  # Accessing a property of the conduit
      # Access other properties like link.roughness, link.inlet_offset, etc.

# Data Gathering

In [ ]:
with Simulation("Inlet_Drains_Model1.inp") as sim:
    links = Links(sim)
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid
      conduit_data = link.conduit_statistics
      peak_flow = conduit_data["peak_flow"]

      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {peak_flow}")

for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  in_offset = conduit_data.offset_upstream # parameter for inlet offset
  out_offset = conduit_data.offset_downstream # parameter for outlet offset

  for xsection_name, xsection_data in xsections.items():
    name = xsection_name
    shape = xsection_data.shape
    geom1 = xsection_data.height # height is geom1 for most shapes
    geom2 = xsection_data.parameter_2 # parameter_2 is geom2 for most shapes
    geom3 = xsection_data.parameter_3 # parameter_3 is geom3 for most shapes
    geom4 = xsection_data.parameter_4 # parameter_4 is geom4 for most shapes
    # To get the idea of Geom 1 to Geom 4, see the guide attached in the Code Documentation in Google Docs

    print (f"Name: {name}")
    print (f"Length: {length} m")
    print (f"Shape: {shape}")
    print (f"Geometry Parameters: {geom1} m, {geom2} m, {geom3} m, {geom4} m")
    print (f"Inlet Offset: {in_offset} m")
    print (f"Outlet Offset: {out_offset} m")
    print (f"")

# Arrange Conduits based on Pipe Flow


In [29]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None):
    """Traces the flow path upstream from a given node, including junctions."""
    if path is None:
        path = []

    path.append(start_node)  # Add the current node (junction or outfall) to the path

    # Find conduits connected to the current node as the downstream
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # If no upstream conduits are found, it's the outermost junction
    if not connected_conduits:
        return path  # Return the path including the outermost junction

    # Trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        new_path = trace_upstream_flow(upstream_node, path + [conduit_name])  # Include conduit in the path
        if new_path:
            return new_path

    return None  # Return None if no path is found (should not happen in a well-defined network)


flow_paths = {}
for outfall_node_name in outfall_node_names:
    flow_path = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = flow_path

# Show results
for outfall_node, flow_path in flow_paths.items():
    flow_path.reverse() # Reverse the flow path list
    print(f"Flow Path for {outfall_node}: {flow_path}")

    # Separate conduits and junctions for clearer output (optional)
    conduits = [item for item in flow_path if item in conduit_connections]
    junctions = [item for item in flow_path if item not in conduit_connections]
    print(f"Sorted Conduits for {outfall_node}: {conduits}")
    print(f"Junctions in Path: {junctions}")

Flow Path for O1: ['Aux3', 'C_Aux3', 'J3', 'C3', 'J4', 'C4', 'J5', 'C5', 'J6', 'C7', 'J8', 'C8', 'J9', 'C9', 'J10', 'C10', 'J11', 'C11', 'O1']
Sorted Conduits for O1: ['C_Aux3', 'C3', 'C4', 'C5', 'C7', 'C8', 'C9', 'C10', 'C11']
Junctions in Path: ['Aux3', 'J3', 'J4', 'J5', 'J6', 'J8', 'J9', 'J10', 'J11', 'O1']


# Identify the Invert Elevations of the Conduits

In [30]:
conduit_info = []
for conduit_name, conduit_data in inp['CONDUITS'].items():
    upstream_junction = conduit_data.from_node
    downstream_junction = conduit_data.to_node

    upstream_invert = junctions_data[upstream_junction].elevation

    # Check if downstream node is an outfall
    if downstream_junction in outfalls_data:
        downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
    elif downstream_junction in junctions_data:
        downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
    else:
        print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
        downstream_invert = None  # or some default value

    length = conduit_data.length

    if downstream_invert is not None:
        slope = (upstream_invert - downstream_invert) / length
    else:
        slope = None
    # Input all data gathered and solve in a DataFrame
    conduit_info.append([conduit_name, upstream_junction, upstream_invert, downstream_junction, downstream_invert, length, slope])

    print(f"Conduit: {conduit_name}; From: {upstream_junction} To: {downstream_junction}")
    print(f"Upstream Invert Elevation: {upstream_invert} m")
    print(f"Downstream Invert Elevation: {downstream_invert} m")
    print(f"Slope: {slope}")

# Exporting DataFrame as an .xlsx File
conduit_df = pd.DataFrame(conduit_info, columns=['Conduit', 'From Node', 'Upstream Invert', 'To Node', 'Downstream Invert', 'Length', 'Slope'])

conduit_df.to_excel("conduit_analysis_data.xlsx", index=False)



Conduit: Street1; From: Aux1 To: Aux2
Upstream Invert Elevation: 4975.0 m
Downstream Invert Elevation: 4973.0 m
Slope: 0.005300681137526172
Conduit: Street2; From: Aux2 To: Aux4
Upstream Invert Elevation: 4973.0 m
Downstream Invert Elevation: 4971.8 m
Slope: 0.0041949241417878
Conduit: Street3; From: Aux4 To: Aux5
Upstream Invert Elevation: 4971.8 m
Downstream Invert Elevation: 4970.7 m
Slope: 0.004594628461636372
Conduit: Street4; From: Aux5 To: Aux6
Upstream Invert Elevation: 4970.7 m
Downstream Invert Elevation: 4969.0 m
Slope: 0.010795021590042026
Conduit: Street5; From: Aux6 To: Aux7
Upstream Invert Elevation: 4969.0 m
Downstream Invert Elevation: 4963.0 m
Slope: 0.011406844106463879
Conduit: C3; From: J3 To: J4
Upstream Invert Elevation: 4973.0 m
Downstream Invert Elevation: 4965.0 m
Slope: 0.07339449541284404
Conduit: C4; From: J4 To: J5
Upstream Invert Elevation: 4965.0 m
Downstream Invert Elevation: 4965.8 m
Slope: -0.00601503759398633
Conduit: C5; From: J5 To: J6
Upstream Inv

Continuity Equation

In [ ]:
# Function for adjusting Pipe Sizes
def adjust_pipe_size (peak_flow, peak_vel):
    Area = peak_flow / peak_vel
    new_diameter = math.sqrt( Area / (0.25 * math.pi))

    pass

# Function: Identifying Peak Flow Rates

In [75]:
def peak_flow_solve(inp, conduit_name):
  with Simulation(inp) as sim:
      links = Links(sim)
      for step in sim:
          pass

      # Iterate through links to find the specific conduit
      for link in links:
          if link.linkid == conduit_name:
              conduit_data = link.conduit_statistics
              peak_flow = conduit_data.get("peak_flow")  # Use get() to handle missing keys
              if peak_flow is not None and peak_flow != 0.0:
                  return float(peak_flow)  # Convert to float if found
              else:
                  return 0.0  # Or handle the missing peak_flow case differently

      # If the conduit is not found
      return 0.0  # Or handle the missing conduit case differently

Test 1

In [31]:
def peak_flow(inp_file_or_object):
    if isinstance(inp_file_or_object, str):
        inp_file = inp_file_or_object  # Use the file path directly
    elif isinstance(inp_file_or_object, SwmmInput):
        # If it's a SwmmInput object, write it to a temporary file
        temp_file_name = "temp_input.inp"
        inp_file_or_object.write_file(temp_file_name)
        inp_file = temp_file_name  # Use the temporary file path
    else:
        raise ValueError("Input must be a file path string or a SwmmInput object")

    with Simulation(inp_file) as sim:
        for step in sim:
            pass

        peak_flow_results = {}
        # Assuming 'inp_file_or_object' has an attribute to access CONDUITS data
        # If not, adjust this part to access the conduit data correctly
        for conduit_name, conduit_data in inp_file_or_object["CONDUITS"].items():
            peak_flow_results[conduit_name] = conduit_data.flow_max

    # If a temporary file was created, delete it along with its outputs
    if isinstance(inp_file_or_object, SwmmInput):
        os.remove(temp_file_name)
        os.remove("temp_input.out") # Delete temporary output file
        os.remove("temp_input.rpt") # Delete temporary report file

    return peak_flow

# Function: Identifying Suitable Pipe Sizes

In [87]:
def find_suitable_sizes(conduit_name, available_sizes, peak_flow):
  if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
    original_size = inp["XSECTIONS"][conduit_name].height  # Adjust based on your inp structure

    if peak_flow == 0:
      return [original_size]  # Return original size as the only suitable size

    suitable_sizes = []
    with open(available_sizes) as file:
      for line in file:
        size = float(line.strip())
        # Changed '^' to '**' for exponentiation
        if peak_flow != 0:  # Add a check for peak_flow
            velocity = peak_flow / (math.pi * size**2 * 0.25)
            if vel_min <= velocity <= vel_max:
                suitable_sizes.append(size)

    return suitable_sizes

  else:
    original_size = inp["XSECTIONS"][conduit_name].height
    return [original_size]

# Function: Construction Cost

In [ ]:
# Function for the Computation of Construction Costs
def construction_costs (unit_cost, length, size):
  pass

# Function: Identifying the Largest Pipe Size and Minimum Slope

In [34]:
# Identify the largest pipe size available and the minimum slope possible
def largest_size_available(available_sizes):
  largest_size = 0.0
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size > largest_size:
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return largest_size

available_sizes = "Available Sizes.txt"
largest_size = largest_size_available(available_sizes)
print(f"Largest Pipe Size: {largest_size} m")

Largest Pipe Size: 5.90551 m


# Function: Solving for the Max Hydraulic Radius

In [97]:
def get_max_hydraulic_radius(inp, conduit_name):

    with Simulation(inp) as sim:
        links = Links(sim)
        max_hydraulic_radius = 0.0  # Initialize max hydraulic radius

        for step in sim:
            # Iterate through links to find the specific conduit
            for link in links:
                if link.linkid == conduit_name:
                    # Get the flow and depth for the current link and timestep
                    flow = link.flow
                    depth = link.depth

                    # Calculate the hydraulic radius (R = A/P)
                    # Assume circular pipe for this example
                    if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
                        diameter = inp["XSECTIONS"][conduit_name].height

                        # Calculate area and wetted perimeter based on depth
                        if depth >= diameter:  # Full flow
                            area = math.pi * (diameter / 2)**2
                            wetted_perimeter = math.pi * diameter
                        else:  # Partial flow
                            theta = 2 * math.acos(1 - (2 * depth / diameter))
                            area = (diameter**2 / 8) * (theta - math.sin(theta))
                            wetted_perimeter = diameter * theta / 2

                        hydraulic_radius = area / wetted_perimeter

                        # Update max_hydraulic_radius if current value is larger
                        max_hydraulic_radius = max(max_hydraulic_radius, hydraulic_radius)

                    else:
                        # Handle other cross-section shapes if needed
                        pass
                    break  # Exit the inner loop once the conduit is found

    return max_hydraulic_radius

# Function: Manning's Equation

In [ ]:
def adjust_slope_with_mannings(input, conduit_name, new_diameter):
    """
    Adjusts the slope of a conduit in a SWMM input file based on a new diameter
    using Manning's equation.

    Args:
        input_file (str): Path to the SWMM input file (.inp).
        conduit_name (str): Name of the conduit to adjust the slope for.
        new_diameter (float): The new diameter of the conduit.
        mannings_n (float, optional): Manning's roughness coefficient. Defaults to 0.013.

    Returns:
        SwmmInput: The modified SwmmInput object with the adjusted slope.
    """

    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    # Get peak flow rate (Q) for the conduit - you'll need to adapt this part
    # based on how you are getting peak flow rates in your code
    # peak_flow = peak_flow_solve(input_file, conduit_name) # or use your similar function
    # peak_flow = get_peak_flow(modified_inp, conduit_name)
    # peak_flow = current_peak_flows[conduit_name]

    # Calculate cross-sectional area and hydraulic radius for the new diameter
    area = math.pi * (new_diameter / 2)**2
    hydraulic_radius = (new_diameter / 2) / 2  # Assuming full flow for now

    # Calculate the new slope (S) using Manning's equation
    new_slope = (peak_flow * mannings_n / (area * hydraulic_radius**(2/3)))**2

    # Return the modified SwmmInput object
    return new_slope

# Function: Identifying Slopes

In [ ]:
def adjusting_slopes(inp, conduit_name, pipe_size)
  new_upstream_invert = {}
  new_downstream_invert = {}

  if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR"
    for conduit_name, conduit_data in inp["CONDUITS"].items():
      upstream_junction = conduit_data.from_node
      downstream_junction = conduit_data.to_node

      upstream_invert = junctions_data[upstream_junction].elevation

      # Check if downstream node is an outfall
      if downstream_junction in outfalls_data:
          downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
      elif downstream_junction in junctions_data:
          downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
      else:
          print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
          downstream_invert = None  # or some default value

Optimization of Pipe Sizes and Slopes

Optimization Test 1

In [ ]:
# Get initial peak flow rates (using your peak_flow_rates function or similar)
initial_peak_flows = peak_flow(rpt)  # Replace with your actual function call

# Define velocity constraints
vel_min = 0.8
vel_max = 5

# Function to find suitable sizes for a given conduit and peak flow
def find_suitable_sizes(conduit_name, peak_flow):
    suitable_sizes = []
    with open("Available Sizes.txt") as file:  # Assuming your available sizes are in this file
        for line in file:
            size = float(line.strip())
            velocity = peak_flow / (math.pi * size**2 * 0.25)
            if vel_min <= velocity <= vel_max:
                suitable_sizes.append(size)
    return suitable_sizes

# Optimization loop
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]

    # Optimize starting pipes simultaneously
    starting_pipes = [conduits_in_path[0]]  # Get the first conduit in the path

    suitable_sizes_per_pipe = {pipe: find_suitable_sizes(pipe, initial_peak_flows[pipe]) for pipe in starting_pipes}
    all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in starting_pipes]))

    for size_combination in all_size_combinations:
        modified_inp = inp.copy()

        # Modify pipe sizes for all starting pipes
        for i, pipe in enumerate(starting_pipes):
            modified_inp["CONDUITS"][pipe].height = size_combination[i]

        # Write modified input file and run simulation
        modified_inp.write_file("modified_input.inp")
        with Simulation("modified_input.inp") as sim:
            for step in sim:
                pass

        # Get updated peak flow rates for next conduits in the series
        # ... (use rpt object to get peak flow rates for downstream conduits) ...

        # Iterate downstream and optimize remaining conduits
        # ... (use similar logic as for starting pipes, but with updated peak flow rates) ...

Optimization Test 2

In [ ]:
# suitable_sizes = find_suitable_sizes(conduit_name, available_sizes, peak_flow)

# Optimization loop
# Get all starting pipes from all flow paths
all_starting_pipes = []
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]
    all_starting_pipes.append(conduits_in_path[0])

# Run initial simulation to get initial peak flows
initial_peak_flows = peak_flow("Inlet_Drains_Model1.inp")

# Get suitable sizes for all starting pipes
suitable_sizes_per_pipe = {
    pipe: find_suitable_sizes(pipe, initial_peak_flows[pipe])
    for pipe in all_starting_pipes
}

# Generate all size combinations for all starting pipes
all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]))

# Optimization for all starting pipes simultaneously
for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        modified_inp["CONDUITS"][pipe].height = size_combination[i]

    # Save modified inp to a separate file
    file_name = f"modified_input_{'_'.join(map(str, size_combination))}.inp"
    modified_inp.write_file(file_name)

    # Run simulation for the saved inp file
    updated_peak_flows = peak_flow(file_name)

    # ... (use updated_peak_flows to get peak flow rates for downstream conduits) ...

    # Iterate downstream and optimize remaining conduits
    # ... (use similar logic as for starting pipes, but with updated peak flow rates) ...


Optimization Test 3

In [ ]:
# Optimization loop
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]

    # --- Start with initial simulation ---
    current_peak_flows = peak_flow(inp)  # Assuming initial input file name
    modified_inp = copy.deepcopy(inp)

    # --- Iterate through conduits in the path ---
    for i, conduit_name in enumerate(conduits_in_path):
        suitable_sizes = find_suitable_sizes(conduit_name, available_sizes, current_peak_flows[conduit_name])

        # --- Try each suitable size for the current conduit ---
        for size in suitable_sizes:
            modified_inp["CONDUITS"][conduit_name].height = size

            # Save modified inp to a separate file
            file_name = f"modified_input_{conduit_name}_{size}.inp"
            modified_inp.write_file(file_name)

            # Run simulation for the saved inp file
            # current_peak_flows = peak_flow(file_name)  # Update peak flows for next conduit
            # Use the SwmmInput object created from the modified file
            updated_inp = SwmmInput(file_name)  # Create SwmmInput object for the modified file
            current_peak_flows = peak_flow(updated_inp)  # Update peak flows

            # ... (You can add logic here to compare results or store data) ...

        # --- After trying all sizes for current conduit, move to the next ---

    # --- After processing all conduits in the path, move to the next flow path ---

TypeError: 'float' object is not callable

Optimization Test 4

In [ ]:
for conduit_name, conduit_data in inp["CONDUITS"].items():
    conduit_data.height = largest_size

# Saving data before Simulation
inp.write_file("Modified_Inlet_Drains_Model1.inp")

# Get all starting pipes from all flow paths
all_starting_pipes = []
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]
    all_starting_pipes.append(conduits_in_path[0])

# Run initial simulation to get initial peak flows
# Changed 'peak_flow' to its original function name
initial_peak_flows = peak_flow(inp) # or use whatever name you originally called your peak flow rates function

# Get suitable sizes for all starting pipes
suitable_sizes_per_pipe = {
    pipe: find_suitable_sizes(pipe, available_sizes, initial_peak_flows[pipe])
    for pipe in all_starting_pipes
}

# Generate all size combinations for all starting pipes
all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]))
print("All size combinations:", all_size_combinations)

# Optimization loop for each size combination
for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        modified_inp["CONDUITS"][pipe].height = size_combination[i]

    # --- Iterate downstream and optimize remaining conduits ---
    current_peak_flows = peak_flow(modified_inp)  # Get peak flows for the modified input

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        for i, conduit_name in enumerate(conduits_in_path):
            # Skip starting pipes as they are already set
            if conduit_name in all_starting_pipes:
                continue

            suitable_sizes = find_suitable_sizes(conduit_name, current_peak_flows[conduit_name])

            # Find best size for current conduit based on cost (or other criteria)
            best_size_for_conduit = suitable_sizes[0]  # You might need a more sophisticated cost function here

            # Update the modified input with the best size
            modified_inp["CONDUITS"][conduit_name].height = best_size_for_conduit

            current_peak_flows = peak_flow(modified_inp)  # Update peak flows for next conduit

    # Save the fully optimized input to a separate file
    file_name = f"optimized_input_{'_'.join(map(str, size_combination))}.inp"
    modified_inp.write_file(file_name)
    print(f"Optimized input file saved as {file_name}")

TypeError: 'function' object is not subscriptable

# Step 1: Initialize All Pipe Sizes and Slopes

In [ ]:
for conduit_name, conduit_data in inp["CONDUITS"].items():
    modified_inp = copy.deepcopy(inp)
    for xsection_name, xsection_data in xsections.items():
      # Get the xsection name associated with the current conduit
      # The attribute is called 'xsection' and not 'xsection_name'
      conduit_id = xsection_name

      # Modify the xsection data in the 'XSECTIONS' section
      xsection_data = modified_inp["XSECTIONS"][xsection_name]
      if callable(largest_size_available):
        conduit_size = largest_size_available(available_sizes)
      else:
        conduit_size = largest_size_available

      if xsection_data.shape == "CIRCULAR":
        xsection_data.height = conduit_size
      else:
        pass

      print(f"Name: {conduit_id}")
      print(f"Shape: {xsection_data.shape}")
      print(f"Conduit Height: {xsection_data.height}")
      print(f"")
      # Create a unique filename to avoid overwriting

file_name = f"Modified_Model1.inp"
modified_inp.write_file(file_name)


# Step 2: Optimize Pipe Sizes

Starting Pipe Size Optimization

In [ ]:
# Get all starting pipes from all flow paths
all_starting_pipes = []
for outfall_node, flow_path in flow_paths.items():
    conduits_in_path = [item for item in flow_path if item in conduit_connections]
    all_starting_pipes.append(conduits_in_path[0])

# Create a dictionary to store initial peak flows for each conduit
initial_peak_flows = {}
for conduit_name in all_starting_pipes:  # or inp['CONDUITS'] to check all
    peak_flow_value = peak_flow_solve("Modified_Model1.inp", conduit_name)
    initial_peak_flows[conduit_name] = peak_flow_value
    print(initial_peak_flows)

# Get suitable sizes for all starting pipes
suitable_sizes_per_pipe = {
    conduit_name: find_suitable_sizes(conduit_name, available_sizes, initial_peak_flows[conduit_name])
    for conduit_name in all_starting_pipes
}
print(suitable_sizes_per_pipe)

# Generate all size combinations for all starting pipes
all_size_combinations = list(itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]))

# Optimization loop for each size combination
for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        # Find the xsection name associated with the pipe
        xsection_name = None
        for xs_name, xs_data in xsections.items():
            if xs_data.link == pipe:
                xsection_name = xs_name
                break

    if xsection_name is not None:
        modified_inp["XSECTIONS"][xsection_name].height = size_combination[i]

    # *** Add this section to save the modified input file ***
    file_name = f"modified_input_starting_pipes_{'_'.join(map(str, size_combination))}.inp"
    modified_inp.write_file(file_name)
    print(f"Modified input file saved as {file_name}")


# #Iterate through all conduits, not just starting pipes
# for xsection_name, xsection_data in xsections.items():
#     pipe_name = xsection_name
#     flow = peak_flow_solve("Modified_Model1.inp", pipe_name)
#     print(f"Pipe Name: {pipe_name}")
#     print(f"Flow: {flow}")
#     print(f"")

    # # Check if flow is not None before proceeding
    # if flow is not None:
    #     suitable_sizes = find_suitable_sizes(pipe_name, available_sizes, flow)
    #     # ... (Rest of your logic to use suitable_sizes) ...
    # else:
    #     print(f"Warning: Could not get peak flow for conduit {conduit_name}. Skipping.")

Downstream Optimization

In [94]:
# Optimization loop for each size combination
for size_combination in all_size_combinations:
    modified_inp = copy.deepcopy(inp)

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        # Find the xsection name associated with the pipe
        xsection_name = None
        for xs_name, xs_data in xsections.items():
            if xs_data.link == pipe:
                xsection_name = xs_name
                break

        if xsection_name is not None:
            modified_inp["XSECTIONS"][xsection_name].height = size_combination[i]

    # --- Iterate downstream and optimize remaining conduits ---
    # Get initial peak flows for this combination using the original peak_flow_solve function
    current_peak_flows = {}
    for conduit_name, _ in modified_inp["CONDUITS"].items():
        # Create a temporary input file
        temp_file_name = f"temp_input_{conduit_name}.inp"
        modified_inp.write_file(temp_file_name)

        # Call peak_flow_solve with the temporary file
        peak_flow_value = peak_flow_solve(temp_file_name, conduit_name)

        # Remove the temporary file
        os.remove(temp_file_name)
        os.remove(temp_file_name.replace(".inp", ".out"))
        os.remove(temp_file_name.replace(".inp", ".rpt"))

        current_peak_flows[conduit_name] = peak_flow_value


    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        for i, conduit_name in enumerate(conduits_in_path):
            # Skip starting pipes as they are already set
            if conduit_name in all_starting_pipes:
                continue

            suitable_sizes = find_suitable_sizes(conduit_name, available_sizes, current_peak_flows[conduit_name])

            # Find best size for current conduit (consider cost or other criteria)
            best_size_for_conduit = suitable_sizes[0]  # Placeholder, replace with your logic

            # Find the xsection name associated with the conduit
            xsection_name = None
            for xs_name, xs_data in xsections.items():
                if xs_data.link == conduit_name:
                    xsection_name = xs_name
                    break

            # Update the modified input with the best size
            if xsection_name is not None:
                modified_inp["XSECTIONS"][xsection_name].height = best_size_for_conduit

            # Update peak flows for the next conduit in the path
            # Get updated peak flows using the original peak_flow_solve function
            current_peak_flows = {}
            for conduit_name, _ in modified_inp["CONDUITS"].items():
                # Create a temporary input file
                temp_file_name = f"temp_input_{conduit_name}.inp"
                modified_inp.write_file(temp_file_name)

                # Call peak_flow_solve with the temporary file
                peak_flow_value = peak_flow_solve(temp_file_name, conduit_name)

                # Remove the temporary file
                os.remove(temp_file_name)
                os.remove(temp_file_name.replace(".inp", ".out"))
                os.remove(temp_file_name.replace(".inp", ".rpt"))

                current_peak_flows[conduit_name] = peak_flow_value

    # Save the fully optimized input to a separate file with a unique name
    file_name = f"optimized_input_starting_pipes_{'_'.join(map(str, size_combination))}_downstream.inp"
    modified_inp.write_file(file_name)
    print(f"Optimized input file saved as {file_name}")

Optimized input file saved as optimized_input_starting_pipes_3.0_downstream.inp


# Step 3: Adjust Pipe Slopes Based on Adjusted Pipe Sizes

In [ ]:
# Get a list of all optimized input files from Step 2
optimized_input_files = [f for f in os.listdir() if f.startswith("optimized_input_starting_pipes_")]

# Iterate through each optimized input file
for input_file in optimized_input_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    # --- Modify conduit slopes/inverts here ---
    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Process conduits in reverse order (from last to first)
        for conduit_name in reversed(conduits_in_path):
            conduit_data = modified_inp["CONDUITS"][conduit_name]

            # Get optimized size
            optimized_size = conduit_data.height

            # Get upstream and downstream node names
            upstream_node = conduit_data.from_node
            downstream_node = conduit_data.to_node

            # Calculate new invert elevations based on optimized size and other factors
            # ... (Your logic to calculate new upstream and downstream inverts) ...
            # You might need to consider the slope, length, and existing inverts
            # of connected conduits to maintain continuity.

            # Update the invert elevations in the JUNCTIONS section
            modified_inp["JUNCTIONS"][upstream_node].elevation = new_upstream_invert

            # Check if downstream node is an outfall, if not, update its invert elevation
            if downstream_node not in modified_inp["OUTFALLS"]:
              modified_inp["JUNCTIONS"][downstream_node].elevation = new_downstream_invert

    # Save the modified input file with a new name
    modified_inp.write_file(f"adjusted_{input_file}")

    print(f"Adjusted slopes/inverts for {input_file} and saved as adjusted_{input_file}")